## Build Models with PyTorch
PyTorch로 모델 만들기

In [1]:
import torch

class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

tinymodel = TinyModel()

print('The model :')
print(tinymodel)

print('\n\nJust one layer :')
print(tinymodel.linear2)

print('\n\nModel params :')
for param in tinymodel.parameters():
    print(param)

print('\n\nLayer params :')
for param in tinymodel.linear2.parameters():
    print(param)

The model :
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Just one layer :
Linear(in_features=200, out_features=10, bias=True)


Model params :
Parameter containing:
tensor([[-0.0031, -0.0222,  0.0984,  ..., -0.0924, -0.0979, -0.0472],
        [-0.0567,  0.0364, -0.0484,  ..., -0.0287,  0.0276,  0.0179],
        [ 0.0420, -0.0620,  0.0159,  ..., -0.0274, -0.0179,  0.0568],
        ...,
        [-0.0843,  0.0510,  0.0729,  ...,  0.0767, -0.0404, -0.0163],
        [-0.0363, -0.0474,  0.0961,  ..., -0.0794, -0.0197,  0.0905],
        [ 0.0785, -0.0861, -0.0437,  ..., -0.0634, -0.0240,  0.0320]],
       requires_grad=True)
Parameter containing:
tensor([ 2.2394e-02, -9.9673e-03,  4.2864e-02, -9.0315e-02,  4.0805e-02,
         7.2054e-02,  1.7982e-02,  3.2671e-02,  5.1330e-02, -1.2510e-03,
        -2.1270e-02, -1.3390e-02,  5.3449e-02, -7.7

## Common Layer Types

### Linear Layers

In [2]:
lin = torch.nn.Linear(3, 2)
x = torch.rand(1, 3)
print('Input :')
print(x)

print('\n\nWeight and Bias parameters :')
for param in lin.parameters():
    print(param)

y = lin(x)
print('\n\nOutput :')
print(y)

Input :
tensor([[0.3724, 0.1462, 0.6992]])


Weight and Bias parameters :
Parameter containing:
tensor([[ 0.3786, -0.4462,  0.1426],
        [ 0.4345,  0.3068,  0.1365]], requires_grad=True)
Parameter containing:
tensor([-0.1608,  0.2749], requires_grad=True)


Output :
tensor([[0.0147, 0.5770]], grad_fn=<AddmmBackward0>)


### Convolutional Layers

In [3]:
import torch.functional as F

class LeNet(torch.nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel (black & white), 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = torch.nn.Conv2d(1, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = torch.nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

### Recurrent Layers

In [4]:
class LSTMTagger(torch.nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim = 1)
        return tag_scores

### Transformers

## Other Layers and Functions

### Data Manipulation Layers

In [5]:
my_tensor = torch.rand(1, 6, 6)
print(my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3)
print(maxpool_layer(my_tensor))

tensor([[[0.5947, 0.9470, 0.8487, 0.4166, 0.4971, 0.4190],
         [0.0381, 0.7499, 0.8456, 0.5935, 0.0706, 0.1089],
         [0.9596, 0.4914, 0.5083, 0.3930, 0.5609, 0.8478],
         [0.2585, 0.4148, 0.5637, 0.9527, 0.2420, 0.0477],
         [0.6764, 0.1093, 0.5605, 0.9452, 0.5564, 0.4071],
         [0.3485, 0.5474, 0.9384, 0.9443, 0.6939, 0.1578]]])
tensor([[[0.9596, 0.8478],
         [0.9384, 0.9527]]])


In [6]:
my_tensor = torch.rand(1, 4, 4) * 20 + 5
print(my_tensor)

print(my_tensor.mean())

norm_layer = torch.nn.BatchNorm1d(4)
normed_tensor = norm_layer(my_tensor)
print(normed_tensor)

print(normed_tensor.mean())

tensor([[[ 7.7811,  7.7007,  7.5060, 20.2949],
         [15.9749,  9.0530,  8.0327, 23.8550],
         [21.2256, 13.4689, 18.6433, 22.8768],
         [16.6556, 12.4958,  5.6967, 18.9211]]])
tensor(14.3864)
tensor([[[-0.5556, -0.5703, -0.6059,  1.7318],
         [ 0.2753, -0.8161, -0.9770,  1.5178],
         [ 0.6101, -1.5688, -0.1153,  1.0739],
         [ 0.6387, -0.1881, -1.5397,  1.0891]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(4.4703e-08, grad_fn=<MeanBackward0>)


In [7]:
my_tensor = torch.rand(1, 4, 4)

dropout = torch.nn.Dropout(p = 0.4)
print(dropout(my_tensor))
print(dropout(my_tensor))

tensor([[[0.0000, 0.2199, 0.1694, 0.0000],
         [0.1455, 0.0000, 1.1674, 1.5979],
         [0.0000, 0.2470, 0.0000, 0.0000],
         [1.3426, 0.1901, 0.0000, 1.4240]]])
tensor([[[0.7978, 0.2199, 0.1694, 1.3536],
         [0.0000, 0.8682, 1.1674, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [1.3426, 0.1901, 0.0000, 1.4240]]])


### Activation Functions
### Loss Functions